In [3]:
import plotly.express as px
import pandas as pd
import sqlite3
import plotly.graph_objects as go

In [22]:
import os
print(os.path.exists("../food_tracker.db"))

True


In [ ]:
def location_nutrient_breakdown (uid):
    # this would work best if only used for the week/month

    conn = sqlite3.connect('../food_tracker.db')

    c = conn.cursor()
    c.execute("SELECT SUM(protein), SUM(fats), SUM(carbohydrates), location_id FROM food_log WHERE uid = ? GROUP BY location_id", (uid, ))

    rows = c.fetchall()

    df = pd.DataFrame(rows, columns=['Protein (g)', 'Fats (g)', 'Carbs (g)', 'Dining Hall'])

    df_long = pd.melt(
        df,
        id_vars=['Dining Hall'],
        value_vars=['Protein (g)', 'Fats (g)', 'Carbs (g)'],
        var_name='Nutrient',
        value_name='Amount'
    )

    fig = px.bar_polar(df_long, r="Amount", theta="Dining Hall", color="Nutrient", template="plotly_dark",
                color_discrete_sequence=["#EF476F", "#FFD166", "#06D6A0"])
    fig.show()

    c.close()
    conn.close()

In [42]:
location_nutrient_breakdown('dg109@wellesley.edu')

In [12]:
def common_dining(uid):
    conn = sqlite3.connect('../food_tracker.db')

    c = conn.cursor()
    c.execute("SELECT location_id FROM food_log WHERE uid = ?", (uid, ))

    rows = c.fetchall()

    df = pd.DataFrame(rows, columns=["Dining Hall"])
    counts = df["Dining Hall"].value_counts().reset_index()
    counts.columns = ["Dining Hall", "# of Visits"]

    # 3. Sort in descending order
    counts = counts.sort_values("# of Visits", ascending=False)

    # 4. Plot horizontal bar chart
    fig = px.bar(
        counts,
        x="# of Visits",
        y="Dining Hall",
        orientation='h'  # horizontal
    )

    fig.update_layout(yaxis=dict(categoryorder='total ascending'))  # most common at top
    fig.show()

In [13]:
common_dining('dg109@wellesley.edu')

In [ ]:
def calorie_goal(uid):
    conn = sqlite3.connect(DB_PATH)
    c = conn.cursor()
    
    c.execute("SELECT SUM(calories), calorie_goal FROM food_log WHERE uid = ?", (uid,))
    row = c.fetchone()
    conn.close()

    if row is None or row[0] is None:
        print("No data found.")
        return

    consumed = row[0]
    goal = row[1]

    if consumed < goal:
        labels = ['Calories Consumed', 'Remaining']
        values = [consumed, goal - consumed]
        colors = ['#FFA07A', '#90EE90']
    elif consumed == goal:
        labels = ['Calories Consumed']
        values = [goal]
        colors = ['#FFA07A']
    else:
        labels = ['Calorie Goal', 'Over Limit']
        values = [goal, consumed - goal]
        colors = ['#FFA07A', '#FF6347']

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, marker=dict(colors=colors))])
    fig.update_traces(textinfo='label+percent')
    fig.update_layout(title_text=f"Calorie Tracker: {consumed:.0f} / {goal:.0f} kcal")
    return fig